In [1]:
import numpy as np
from scipy import io as sio
from matplotlib import pyplot as plt
import inspect

In [2]:
# Define a function to see the source code of a function. 
def showfunc(functiontoshow):
    print(inspect.getsource(functiontoshow))

In [3]:
import typhon as tp

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Set environmental variables. 
tp.environ['ARTS_BUILD_PATH'] = '../../arts-dev/build/'
tp.environ['ARTS_INCLUDE_PATH'] = '../../arts-dev/controlfiles/'

In [5]:
# Import Typhon retrieval packages. 
import typhon.retrieval.bmci as tprb
import typhon.retrieval.mcmc as tprm
import typhon.retrieval.qrnn as tprq

Loading ARTS API from: ../../arts-dev/build/src/libarts_api.so


Using TensorFlow backend.


In [6]:
# Create an ARTS workspace variable. 
ws = tp.arts.workspace.Workspace()

In [29]:
%%capture
# ARTS INCLUDE files
ws.execute_controlfile("general/general.arts");
ws.execute_controlfile("general/continua.arts");
ws.execute_controlfile("general/agendas.arts");
ws.execute_controlfile("general/planet_earth.arts");

In [14]:
showfunc(ws.abs_xsec_agenda.)

TypeError: ARTS Workspace Variable

Name:  abs_xsec_agenda
Group: Agenda

Agenda to calculate scalar gas absorption cross sections.
 is not a module, class, method, function, traceback, frame, or code object

In [ ]:
# Load the jacobian calculated from the forward model. 
J = tp.arts.xml.load("./ClearSky_1D_jacobian.xml")
J.shape

In [43]:
# ARTS test run: 
tp.arts.run_arts(controlfile='./ClearSky_1D_OEM_ARTSvdev.arts')

Exception: ARTS run failed:
arts: /home/user/Desktop/ARTS/arts-dev/src/matpackI.cc:164: Range::Range(const Range &, const Range &): Assertion `fin <= prev_fin' failed.


In [ ]:
Tb_simulated = tp.arts.xml.load("./ClearSky_1D_Tb.xml")
Tb_BosungObs = np.array([26.193, 23.754, 22.435, 18.510, 15.617, 12.566, 12.388, 11.472, 
                         107.09, 123.677, 147.536, 178.933, 216.787, 250.225, 273.193, 282.353, 
                         286.576, 288.430, 289.69, 290.449, 291.361, 291.436])